### Analysis description
Here we aim at drawing all pycortex flatmaps and create webgl.<br/>
This analysis is ran first with _task = 'FullScreen'_ to determine and _save_svg_ = True.<br/>
Use _show_plot = True_ to inspect flatmaps.<br/>

In [ ]:
# Imports
import os
import numpy as np
import nibabel as nb
import shutil
import warnings
import cortex
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib as mpl
warnings.filterwarnings('ignore')

# Define parameters
subjects = ['sub-001','sub-002','sub-003','sub-004',
            'sub-005','sub-006','sub-007','sub-008']
tasks = ['FullScreenAttendFix', 
         'FullScreenAttendBar',
         'FullScreen']

# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)
pycortex_dir = "{}/cortex".format(pp_dir)

# Plot parameters
cmap_polar, cmap_uni, cmap_ecc_size, cmap_2d = 'hsv', 'Reds', 'Spectral', 'PU_RdBu_covar_alpha'
save_svg = False
show_plot = False
xfm_name = 'identity.fmriprep'
amp_th = 0
r2_th = [0.05, 1]
ecc_th = [0.1, 20]
sd_th = [0.1, 20]
alpha_range = [0, 0.4]
r2_range = [0, 1]
r2_2d_range = [0, 0.4]
ecc_range = [0, 10]
size_range = [0, 10]
col_offset = 1.0/14.0
cmap_steps = 255
recache = False

In [ ]:
# Write pycortex config file
pycortex_db = "{}/db/".format(pycortex_dir)
pycortex_cm = "{}/colormaps/".format(pycortex_dir)
pycortex_config_file  = cortex.options.usercfg
pycortex_config_file_new = pycortex_config_file[:-4] + '_new.cfg'
pycortex_config_file_old = pycortex_config_file[:-4] + '_old.cfg'

Path(pycortex_config_file_new).touch()
with open(pycortex_config_file, 'r') as fileIn:
    with open(pycortex_config_file_new, 'w') as fileOut:
        for line in fileIn:
            if 'filestore' in line:
                newline = 'filestore=' + pycortex_db
                fileOut.write(newline)
                newline = '\n'
            elif 'colormaps' in line:
                newline = 'colormaps=' + pycortex_cm
                fileOut.write(newline)
                newline = '\n'
            else:
                newline = line
            fileOut.write(newline)
os.rename(pycortex_config_file,pycortex_config_file_old)
os.rename(pycortex_config_file_new, pycortex_config_file)

In [ ]:
def draw_cortex(subject, xfmname, data, vmin, vmax, description, cortex_type='VolumeRGB',
                cmap='Viridis', cbar='discrete', cmap_steps=255,
                alpha=None, depth=1, thick=1, height=1024, sampler='nearest',
                with_curvature=True, with_labels=False, with_colorbar=False,
                with_borders=False, curv_brightness=0.95, curv_contrast=0.05, add_roi=False,
                roi_name='empty', col_offset=0, cbar_label=''):
    """
    Plot brain data onto a previously saved flatmap.
    Parameters
    ----------
    subject             : subject id (e.g. 'sub-001')
    xfmname             : xfm transform
    data                : the data you would like to plot on a flatmap
    cmap                : colormap that shoudl be used for plotting
    vmins               : minimal values of 1D 2D colormap [0] = 1D, [1] = 2D
    vmaxs               : minimal values of 1D/2D colormap [0] = 1D, [1] = 2D
    description         : plot title
    cortex_type         : cortex function to create the volume (VolumeRGB, Volume2D)
    cbar                : color bar layout
    cbar_label          : colorbar label
    cmap_steps          : number of colormap bins
    alpha               : alpha map or dim2
    depth               : Value between 0 and 1 for how deep to sample the surface for the flatmap (0 = gray/white matter boundary, 1 = pial surface)
    thick               : Number of layers through the cortical sheet to sample. Only applies for pixelwise = True
    height              : Height of the image to render. Automatically scales the width for the aspect of the subject's flatmap
    sampler             : Name of sampling function used to sample underlying volume data. Options include 'trilinear', 'nearest', 'lanczos'
    with_curvature      : Display the rois, labels, colorbar, annotated flatmap borders, or cross-hatch dropout?
    with_labels         : Display labels?
    with_colorbar       : Display pycortex' colorbar?
    with_borders        : Display borders?
    curv_brightness     : Mean brightness of background. 0 = black, 1 = white, intermediate values are corresponding grayscale values.
    curv_contrast       : Contrast of curvature. 1 = maximal contrast (black/white), 0 = no contrast (solid color for curvature equal to curvature_brightness).
    add_roi             : add roi -image- to overlay.svg
    roi_name            : roi name
    col_offset          : colormap offset between 0 and 1

    Returns
    -------
    braindata - pycortex volume file
    """
    
    if cortex_type=='VolumeRGB':
        # define colormap
        try: base = plt.cm.get_cmap(cmap)
        except: base = cortex.utils.get_cmap(cmap)

        if '_alpha' in cmap: base.colors = base.colors[1,:,:]
        val = np.linspace(0, 1,cmap_steps+1,endpoint=False)
        colmap = colors.LinearSegmentedColormap.from_list('my_colmap',base(val), N = cmap_steps)
        
        # convert data to RGB
        vrange = float(vmax) - float(vmin)
        norm_data = ((data-float(vmin))/vrange)*cmap_steps
        mat = colmap(norm_data.astype(int))*255.0
        alpha = alpha*255.0

        # define volume RGB
        braindata = cortex.VolumeRGB(channel1=mat[...,0].T.astype(np.uint8), channel2=mat[...,1].T.astype(np.uint8), channel3=mat[...,2].T.astype(np.uint8),
                                     alpha=alpha.T.astype(np.uint8), subject=subject, xfmname=xfmname)
    elif cortex_type=='Volume2D':
        braindata = cortex.Volume2D(dim1=data.T, dim2=alpha.T, subject=subject, xfmname=xfmname, description=description,
                                    cmap=cmap, vmin=vmin[0], vmax=vmax[0], vmin2=vmin[1], vmax2=vmax[1])
    
    braindata_fig = cortex.quickshow(braindata=braindata, depth=depth, thick=thick, height=height, sampler=sampler, with_curvature=with_curvature, with_labels=with_labels, 
                                     with_colorbar=with_colorbar, with_borders=with_borders, curvature_brightness=curv_brightness, curvature_contrast=curv_contrast)

    if cbar == 'polar':
        try: base = plt.cm.get_cmap(cmap)
        except: base = cortex.utils.get_cmap(cmap)
        val = np.arange(1,cmap_steps+1)/cmap_steps - (1/(cmap_steps*2))
        val = np.fmod(val+col_offset,1)
        colmap = colors.LinearSegmentedColormap.from_list('my_colmap', base(val), N=cmap_steps)
        cbar_axis = braindata_fig.add_axes([0.5, 0.07, 0.8, 0.2], projection='polar')
        norm = colors.Normalize(0, 2*np.pi)
        t = np.linspace(0,2*np.pi,200,endpoint=True)
        r = [0,1]
        rg, tg = np.meshgrid(r,t)
        im = cbar_axis.pcolormesh(t, r, tg.T,norm=norm, cmap=colmap)
        cbar_axis.set_yticklabels([])
        cbar_axis.set_xticklabels([])
        cbar_axis.set_theta_zero_location("W")
        cbar_axis.spines['polar'].set_visible(False)

    elif cbar == 'ecc':
        colorbar_location = [0.5, 0.07, 0.8, 0.2]
        n = 200
        cbar_axis = braindata_fig.add_axes(colorbar_location, projection='polar')
        t = np.linspace(0,2*np.pi, n)
        r = np.linspace(0,1, n)
        rg, tg = np.meshgrid(r,t)
        c = tg
        im = cbar_axis.pcolormesh(t, r, c, norm = mpl.colors.Normalize(0, 2*np.pi), cmap=colmap)
        cbar_axis.tick_params(pad=1,labelsize=15)
        cbar_axis.spines['polar'].set_visible(False)
        box = cbar_axis.get_position()
        cbar_axis.set_yticklabels([])
        cbar_axis.set_xticklabels([])
        axl = braindata_fig.add_axes([0.97*box.xmin,0.5*(box.ymin+box.ymax), box.width/600,box.height*0.5])
        axl.spines['top'].set_visible(False)
        axl.spines['right'].set_visible(False)
        axl.spines['bottom'].set_visible(False)
        axl.yaxis.set_ticks_position('right')
        axl.xaxis.set_ticks_position('none')
        axl.set_xticklabels([])
        axl.set_yticklabels(np.linspace(vmin, vmax, 3),size = 'x-large')
        axl.set_ylabel('$dva$\t\t', rotation=0, size='x-large')
        axl.yaxis.set_label_coords(box.xmax+30,0.4)
        axl.patch.set_alpha(0.5)

    elif cbar == 'discrete':
        colorbar_location= [0.8, 0.05, 0.1, 0.05]
        cmaplist = [colmap(i) for i in range(colmap.N)]
        bounds = np.linspace(vmin, vmax, cmap_steps + 1)  
        bounds_label = np.linspace(vmin, vmax, 3)
        norm = mpl.colors.BoundaryNorm(bounds, colmap.N)
        
        cbar_axis = braindata_fig.add_axes(colorbar_location)
        cb = mpl.colorbar.ColorbarBase(cbar_axis, cmap=colmap, norm=norm, ticks=bounds_label, boundaries=bounds,orientation='horizontal')
        cb.set_label(cbar_label,size='x-large')
        
    elif cbar == '2D':
        cbar_axis = braindata_fig.add_axes([0.8, 0.05, 0.15, 0.15])
        base = cortex.utils.get_cmap(cmap)
        cbar_axis.imshow(np.dstack((base.colors[...,0], base.colors[...,1], base.colors[...,2],base.colors[...,3])))
        cbar_axis.set_xticks(np.linspace(0,255,3))
        cbar_axis.set_yticks(np.linspace(0,255,3))
        cbar_axis.set_xticklabels(np.linspace(vmin[0],vmax[0],3))
        cbar_axis.set_yticklabels(np.linspace(vmax[1],vmin[1],3))
        cbar_axis.set_xlabel(cbar_label[0], size='x-large')
        cbar_axis.set_ylabel(cbar_label[1], size='x-large')
        
    if add_roi == True:
        cortex.utils.add_roi(data=braindata, name=roi_name, open_inkscape=False, add_path=False, depth=depth, thick=thick, sampler=sampler, with_curvature=with_curvature,
                             with_colorbar=with_colorbar, with_borders=with_borders, curvature_brightness=curv_brightness, curvature_contrast=curv_contrast)
    return braindata

In [ ]:
for subject in ['sub-003','sub-004','sub-005','sub-006','sub-007','sub-008']:
    maps_names = []
    volumes = {}
    for task in tasks:
        
        # Define and create folder
        fit_dir = '{}/{}/prf/fit'.format(pp_dir, subject)
        flatmaps_dir = '{}/{}/prf/flatmaps'.format(pp_dir, subject)
        try: os.makedirs(flatmaps_dir) 
        except: pass

        # Load fit parameters
        r2_im = nb.load('{}/{}_task-{}_par-r2.nii.gz'.format(fit_dir,subject,task))
        ecc_im = nb.load('{}/{}_task-{}_par-ecc.nii.gz'.format(fit_dir,subject,task))
        sd_im = nb.load('{}/{}_task-{}_par-sd.nii.gz'.format(fit_dir,subject,task))
        amp_im = nb.load('{}/{}_task-{}_par-amplitude.nii.gz'.format(fit_dir,subject,task))
        x_im = nb.load('{}/{}_task-{}_par-x.nii.gz'.format(fit_dir,subject,task))
        y_im = nb.load('{}/{}_task-{}_par-y.nii.gz'.format(fit_dir,subject,task))

        # Create param matrix
        r2_idx, ecc_idx, sd_idx, amp_idx, x_idx, y_idx = 0, 1, 2, 3, 4, 5
        param_mat = np.zeros((r2_im.shape[0],r2_im.shape[1],r2_im.shape[2],6))*np.nan
        param_mat[...,r2_idx] = r2_im.get_fdata()
        param_mat[...,ecc_idx] = ecc_im.get_fdata()
        param_mat[...,sd_idx] = sd_im.get_fdata()
        param_mat[...,amp_idx] = amp_im.get_fdata()
        param_mat[...,x_idx] = x_im.get_fdata()
        param_mat[...,y_idx] = y_im.get_fdata()

        # Threshold param matrix by masking R2
        param_mat_th = param_mat
        amp_down =  param_mat_th[...,amp_idx] > amp_th
        r2_th_down, r2_th_up = param_mat_th[...,r2_idx] >= r2_th[0],  param_mat_th[...,r2_idx] <= r2_th[1]
        ecc_th_down, ecc_th_up = param_mat_th[...,ecc_idx] >= ecc_th[0], param_mat_th[...,ecc_idx] <= ecc_th[1]
        sd_th_down, sd_th_up = param_mat_th[...,sd_idx] >= sd_th[0], param_mat_th[...,sd_idx] <= sd_th[1]
        all_th = np.array((amp_down, r2_th_down, r2_th_up, ecc_th_down, ecc_th_up, sd_th_down, sd_th_up)) 
        param_mat[np.logical_and.reduce(all_th)==False, r2_idx]=0
        
        # get R2 across tasks
        if task == 'FullScreenAttendFix': 
            r2_attend_mat = np.zeros((r2_im.shape[0],r2_im.shape[1],r2_im.shape[2],2))*np.nan
            r2_attend_mat[...,0] = param_mat[...,r2_idx]
        elif task == 'FullScreenAttendBar': 
            r2_attend_mat[...,1] = param_mat[...,r2_idx]
        
        # Define pRF R2
        r2_data = param_mat[...,r2_idx]
        alpha = (r2_data - alpha_range[0])/(alpha_range[1]-alpha_range[0])
        alpha[alpha>1]=1
        r2_param = {'data': param_mat[...,r2_idx], 'cmap': cmap_uni, 'alpha': alpha, 'vmin': r2_range[0],'vmax': r2_range[1],
                    'cbar': 'discrete', 'cbar_label': 'R2', 'description': '{}: pRF R2'.format(task), 'curv_brightness': 1, 
                    'curv_contrast': 0.1, 'add_roi': False, 'subject': subject, 'xfmname': xfm_name}
        exec('{}_r2_param = r2_param'.format(task))
        maps_names.append('{}_r2'.format(task))
        
        # Define pRF eccentricity
        ecc_data = param_mat[...,ecc_idx]
        ecc_param = {'data': ecc_data, 'cmap': cmap_ecc_size, 'alpha': alpha, 'vmin': ecc_range[0], 'vmax': ecc_range[1],
                     'cbar': 'ecc','roi_name': 'pRF_eccentricity','description': '{}: pRF eccentricity'.format(task), 'curv_brightness': 1, 
                     'curv_contrast': 0.1, 'add_roi': save_svg, 'subject': subject, 'xfmname': xfm_name}
        exec('{}_ecc_param = ecc_param'.format(task))
        maps_names.append('{}_ecc'.format(task))
        
        # Define pRF polar angle
        pol_comp_num = param_mat[...,x_idx] + 1j * param_mat[...,y_idx]
        polar_ang = np.angle(pol_comp_num)
        ang_norm = (polar_ang + np.pi) / (np.pi * 2.0)
        ang_norm = np.fmod(ang_norm + col_offset,1)
        polar_param = {'data': ang_norm, 'cmap': cmap_polar, 'alpha': alpha, 'vmin': 0, 'vmax': 1, 'cmap_steps': cmap_steps, 
                       'cbar': 'polar', 'roi_name': 'pRF_polar_angle', 'col_offset': col_offset, 'description': '{}: pRF polar angle'.format(task), 
                       'curv_brightness': 0.1, 'curv_contrast': 0.25, 'add_roi': save_svg, 'subject': subject, 'xfmname': xfm_name}
        exec('{}_polar_param = polar_param'.format(task))
        maps_names.append('{}_polar'.format(task))
        
        # Define pRF size
        size_data = param_mat[...,sd_idx]
        size_param = {'data': size_data, 'cmap': cmap_ecc_size, 'alpha': alpha, 'vmin': size_range[0], 'vmax': size_range[1], 
                      'cbar': 'discrete', 'cbar_label': 'pRF size (dva)', 'description': '{}: pRF size'.format(task), 'curv_brightness': 1, 
                      'curv_contrast': 0.1, 'add_roi': False, 'subject': subject, 'xfmname': xfm_name}
        exec('{}_size_param = size_param'.format(task))
        maps_names.append('{}_size'.format(task))
        
        # Define R2 attention comparison
        if task == 'FullScreen':
            attcmp_param = {'data':  r2_attend_mat[...,0], 'alpha': r2_attend_mat[...,1], 'cmap': cmap_2d, 'vmin': [r2_2d_range[0],r2_2d_range[0]],
                            'vmax': [r2_2d_range[1],r2_2d_range[1]], 'cortex_type': 'Volume2D', 'cbar': '2D', 'add_roi': False,
                            'description': 'Fullscreen: AttendFix vs. AttendBar', 'curv_brightness': 1, 'curv_contrast': 0.1,
                            'cbar_label': ['AttendFix','AttednBar'], 'subject': subject, 'xfmname': xfm_name}
            exec('{}_attcmp_param = attcmp_param'.format(task))
            maps_names.append('{}_attcmp'.format(task))

    # Flatmaps
    for maps_name in maps_names:
        # Create flatmap
        exec('{}_volume = draw_cortex(**{}_param)'.format(maps_name,maps_name))
        print("{}_task-{}".format(subject, maps_name))
        exec("plt.savefig('{}/{}_task-{}.pdf')".format(flatmaps_dir, subject, maps_name))
        if show_plot: plt.show()
        
        # Save flatmap in dataset
        exec('vol_description = {}_param["description"]'.format(maps_name))
        exec('volume = {}_volume'.format(maps_name))
        volumes.update({vol_description:volume})
    
    # Save Webgl
    webgl_dir = '{}/derivatives/webgl_data/{}/'.format(base_dir, subject)
    if os.path.exists(webgl_dir):
        print('deleting old : {}'.format(webgl_dir))
        shutil.rmtree(webgl_dir)
    print('saving: {}'.format(webgl_dir))
    try: os.makedirs(webgl_dir)
    except: pass
    cortex.webgl.make_static(outpath=webgl_dir, data=volumes, recache=recache)